In [2]:
import random

import openai
import numpy as np
import pandas as pd
from getpass import getpass

from helpers import vignettes_en, validated_codes
from helpers import single_scenario_generation

openai.api_key = getpass()

In [31]:
vignettes_list = vignettes_en.split("\n\n")

scenarios = zip(vignettes_list, validated_codes)

random.seed(828)

scenarios_to_run = random.choices(list(scenarios), k=20, )
# remove duplicates
scenarios_to_run = list(set(scenarios_to_run))

scenarios_to_run

[('You see a man telling his girlfriend that she has to convert to his religion.',
  503),
 ('You see a girl laughing at another student who forgot the lines at a school theatre play.',
  111),
 ('You see a boy leaving a series of booby traps to kill loose cats in his neighborhood.',
  201),
 ('You see a boy throwing stones at the cows that are in the local meadow.',
  206),
 ('You see a boy jump the line because his friend is an employee.', 408),
 ('You see a father forcing his son to become a airplane pilot just like him.',
  501),
 ('You see a trainee disobey a company order to dress professionally and comb his hair.',
  610),
 ('You see a teacher publicly saying that she hopes another school to win the math contest',
  702),
 ('You see a woman intentionally driving her car to run over a squirrel.',
  203),
 ('You see a man saying to his fiancée that she has to change to his political party.',
  509),
 ('You see a woman having sex with her deceased husband.', 808),
 ('You see a boy 

# Pure MFV GPT 3-5

In [ ]:
model = "gpt-3.5-turbo"
for scenario in scenarios_to_run[10:]:
    print(f"Running scenario {scenario[1]}")
    df = single_scenario_generation(scenario, model, 108)

In [ ]:
# reprocessing files
from pathlib import Path
from helpers import process_answer
import json
files = Path("data").glob("raw/raw_results_*_2023-08-23*.json")

for f in files:
    mfv_code = f.stem.split("_")[2]
    print(mfv_code)
    now = f.stem.split("_", maxsplit=4)[-1]
    fl_content = json.load(f.open())
    model = f.stem.split("_")[3]
    answers = list()
    for i, raw in fl_content:
        processed = process_answer(raw)
        if len(processed) != 1:
            print(f"WARNING: Answer {i} has {len(processed)} valid answers")
            continue
        answers.append((i, processed))
    df = pd.DataFrame(answers, columns=["gen_id", "answer"])
    df["mfv_code"] = mfv_code
    df.to_csv(f"data/results_scenario_{mfv_code}_{model}_{now}.csv", index=False)
    
        

# GPT - 4

In [32]:
openai.organization = "org-bduO767V60bsXkNHgnzwStjp"

In [33]:
model = "gpt-4"
for scenario in scenarios_to_run:
    print(f"Running scenario {scenario[1]}")
    df = single_scenario_generation(scenario, model, 108)

Running scenario 503
Running scenario 111


# Draft to replicate study 1

In [ ]:
append_to_system_prompt = """
Furthermore, you must answer why is the action morally wrong? (Select only the main reason.)

    a) It violates norms of harm or care (e.g., unkindness, causing pain to another)

    b) It violates norms of fairness or justice (e.g., cheating or reducing equality)

    c) It violates norms of loyalty (e.g., betrayal of a group)

    d) It violates norms of respecting authority (e.g., subversion, lack of respect for tradition)

    e) It violates norms of purity (e.g., degrading or disgusting acts)

    f) It violates norms of freedom (e.g., bullying, dominating)

    g)It is not morally wrong and does not apply to any of the provided choices"""



In [19]:
import re
import json

import pandas as pd
from pathlib import Path
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponentia

from prompts import system_prompt, alternative_system

rep = Path(".") / "mfv material"

DF_PT = pd.read_excel(rep / "vignettes_pt.xlsx", sheet_name=0)
validated_codes = DF_PT["MFV Code"].to_list()
foundations = DF_PT["Foundation"].values

original_validated = pd.read_html("https://link.springer.com/article/10.3758/s13428-014-0551-2/tables/6")[0]
original_validated["mfv_code"] = [101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,201,202,203,204,205,206,207,208,301,302,303,401,402,403,404,405,406,407,408,409,410,411,412,501,502,503,504,505,506,507,508,509,510,511,601,602,603,604,605,606,607,608,609,610,611,612,613,614,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,801,802,803,804,805,806,807,808,809,810]
# query for validated codes
original_validated.query("mfv_code in @validated_codes", inplace=True)

In [21]:
original_validated.sample(2)

,Foundation,Scenario,mfv_code
17,"Care (p,a)",You see a woman throwing her cat across the ro...,202
13,Care (e),You see a girl telling a boy that his older br...,114


In [24]:
raw_response = "1. 4, b\n2. 2, g"

def run_foundation_experiment(scenarios, codes, model, **kwargs):
    vignettes = "\n".join(scenarios)
    new_sys = system_prompt + append_to_system_prompt
    msg_hist = [
        {"role": "system", "content": new_sys},
        {"role": "user", "content": vignettes},
    ]
    raw_response = chat(
        messages=msg_hist,
        model=model,
        max_tokens=1024,
        temperature=1.2,
        **kwargs,
    )
    p = Path("data/raw/foundation_validation")
    p.mkdir(exist_ok=True)
    with open(p/f"{model}_{now}.json", "w") as f:
        json.dump({"response": raw_response, "scenarios": scenarios, "mfv_codes": codes}, f)
    exp_results = process_judgment(raw_response)
    df = pd.DataFrame(exp_results, columns=["vignette", "foundation"])
    df["mfv_code"] = codes
    return df


def process_judgment(answer, expected=2):
    import re
    pat = r"\d\.\s?(\d+),\s?([a-g])"
    matches = re.findall(pat, answer)
    if len(matches) == 0 or len(matches) != expected:
        print(f"WARNING: Answer has {len(matches)} valid answers")
        return None
    else:
        return matches

In [ ]:
def foundation_validation_generation(n, model):
    results = list()
    
    for i in n:
        df = original_validated.sample(1)